# Web Scraping and Analysis

[NASA Mars Newsite](https://mars.nasa.gov/news/)

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from env import path
from splinter import Browser

In [5]:
#connect to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#define db and collection
db = client.mars_db
collection = db.items

In [6]:
#to execute the path -- victoria's chunk

executable_path = {'executable_path': path}
browser = Browser('chrome', **executable_path, headless=False)
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)
nasa_html = browser.html
nasa_soup=bs(nasa_html, 'html.parser')

In [11]:
def init_browser():
    executable_path = {"executable_path": path}
    return Browser("chrome", **executable_path, headless=False)

browser = init_browser()

In [25]:
scrape_dict={}

In [22]:
titles = []

results = nasa_soup.find_all('div',class_="content_title")

# for result in results:
#     #print(result.text)
#     titles.append(result.text.strip())
    
latest_title=results[0].text
latest_title

scrape_dict["latest_title"] = latest_title

In [23]:
paragraph = []
paras = nasa_soup.find_all('div', class_='rollover_description_inner')
paras


latest_paragraph = paras[0].text
latest_paragraph

scrape_dict["latest_paragraph"] = latest_paragraph

## Mars Featured Image

[Link](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)

In [24]:
featured_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image)
mars_browser = browser.html
featured_mars=bs(mars_browser,'html.parser')

In [25]:
featured_path = featured_mars.find("a","fancybox")["data-fancybox-href"]
jpl_link = "https://www.jpl.nasa.gov/" + featured_path
scrape_dict["featured_link"]= jpl_link

print(jpl_link)


https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA16469_ip.jpg


## Mars Weather

In [26]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
weather_response = requests.get(twitter_url)

weather_tweets = bs(weather_response.text,"html.parser")


In [27]:
#first weather tweet for mars 
latest_weather = weather_tweets.find_all("p",class_="TweetTextSize")[0].text.strip()
latest_weather

scrape_dict["latest_weather"] = latest_weather

latest_weather

'Sol 2026 (April 18, 2018), Sunny, high -6C/21F, low -73C/-99F, pressure at 7.19 hPa, daylight 05:26-17:21'

## Mars Facts

In [28]:
facts_url = "https://space-facts.com/mars/"
facts_response = requests.get(facts_url)
facts_soup = bs(facts_response.text,"html.parser")

In [29]:
table = facts_soup.find_all('table')[0]
table

<table class="tablepress tablepress-id-mars" id="tablepress-mars">
<tbody>
<tr class="row-1 odd">
<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>
</td>
</tr>
<tr class="row-2 even">
<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>
</td>
</tr>
<tr class="row-3 odd">
<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>
</tr>
<tr class="row-4 even">
<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>
</tr>
<tr class="row-5 odd">
<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>
</tr>
<tr class="row-6 even">
<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>
</td>
</tr>
<tr class="row-7 odd">
<td class

In [30]:
facts_list = pd.read_html(str(table))[0]

df = facts_list.rename(index = str, columns={0:"Description", 1:"Facts"})

html_table = df.to_html()

scrape_dict["facts_table"] = html_table

In [31]:
facts_list_al = pd.read_html(str(table))
dict_list = [dict(blank) for blank in facts_list_al]
dict_list

#scrape_dict["dict_list"] = dict_list

[{0: 0    Equatorial Diameter:
  1         Polar Diameter:
  2                   Mass:
  3                  Moons:
  4         Orbit Distance:
  5           Orbit Period:
  6    Surface Temperature:
  7           First Record:
  8            Recorded By:
  Name: 0, dtype: object, 1: 0                         6,792 km
  1                         6,752 km
  2    6.42 x 10^23 kg (10.7% Earth)
  3              2 (Phobos & Deimos)
  4         227,943,824 km (1.52 AU)
  5             687 days (1.9 years)
  6                    -153 to 20 °C
  7                2nd millennium BC
  8             Egyptian astronomers
  Name: 1, dtype: object}]

In [32]:
table

<table class="tablepress tablepress-id-mars" id="tablepress-mars">
<tbody>
<tr class="row-1 odd">
<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>
</td>
</tr>
<tr class="row-2 even">
<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>
</td>
</tr>
<tr class="row-3 odd">
<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>
</tr>
<tr class="row-4 even">
<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>
</tr>
<tr class="row-5 odd">
<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>
</tr>
<tr class="row-6 even">
<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>
</td>
</tr>
<tr class="row-7 odd">
<td class

## Mars Hemispheres

In [39]:
link_pic1 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
cerberus_response = requests.get(link_pic1)
cerberus_soup = bs(cerberus_response.text,"html.parser")
#print(cerberus_soup.prettify())

cerberus_soup.find_all("img",class_="wide-image")
cerberus_img = "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"
print(cerberus_img)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [40]:
link_pic2 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
schiaparelli_response = requests.get(link_pic2)
schiaparelli_soup = bs(schiaparelli_response.text,"html.parser")
#print(schiaparelli_soup.prettify())

schiaparelli_soup.find_all("img",class_="wide-image")
schiaparelli_img = "https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg"

print(schiaparelli_img)


https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [41]:
link_pic3= "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
syrtis_response = requests.get(link_pic3)
syrtis_soup = bs(syrtis_response.text,"html.parser")

syrtis_soup.find_all("img",class_="wide-image")
syrtis_img = "https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg"

print(syrtis_img)

https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [42]:
link_pic4= "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"
valles_response = requests.get(link_pic4)
valles_soup = bs(valles_response.text,"html.parser")

valles_soup.find_all("img",class_="wide-image")
valles_img ="https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg"

print(valles_img)

https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [37]:
hemisphere_image_urls = [
    {"title":"Cerberus Hemisphere","img_url":cerberus_img},
    {"title":"Schiaparelli Hemisphere","img_url":schiaparelli_img},
    {"title":"Syrtis Major","img_url":syrtis_img},
    {"title":"Valles Marineris","img_url":valles_img}
    
]

hemisphere_image_urls

scrape_dict["hemisphere_image_urls"] = hemisphere_image_urls


In [50]:
scrape_dict["hemisphere_image_urls"]

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris'}]

In [53]:
face_links_mars = []
for i in scrape_dict["hemisphere_image_urls"]:
    print(i['img_url'])
    face_links_mars.append(i['img_url'])
    
face_links_mars[0]
    

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [54]:
face_links_mars = []
for i in scrape_dict["hemisphere_image_urls"]:
    face_links_mars.append(i['img_url'])
mf1 = face_links_mars[0]
scrape_dict["mf1"] = mf1
mf2 = face_links_mars[1]
scrape_dict["mf2"] = mf2
mf3 = face_links_mars[2]
scrape_dict["mf3"] = mf3
mf4 = face_links_mars[3]
scrape_dict["mf4"] = mf4

In [55]:
scrape_dict

{'facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Facts</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <t

In [61]:
print(jpl_link)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA16469_ip.jpg


In [58]:
mf1

'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [59]:
cerberus_img

'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [60]:
print(cerberus_img)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [ ]:
featured_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(featured_image)
mars_browser = browser.html
featured_mars=bs(mars_browser,'html.parser')
featured_path = featured_mars.find("a","fancybox")["data-fancybox-href"]
jpl_link = "https://www.jpl.nasa.gov/" + featured_path
scrape_dict["jpl_link"]= jpl_link

In [78]:
browser = init_browser()

fourmars_link = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
browser.visit(fourmars_link)
fourfaces = browser.html
html_fourface = bs(fourfaces,'html.parser')
cer_hem = html_fourface.find("img", class_="wide-image")["src"]
cer_hem



'/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [80]:
astrogeology_url = "https://astrogeology.usgs.gov"
cerberus_png = astrogeology_url + cer_hem
print(cerberus_png)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [83]:
link_pic2 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
browser.visit(link_pic2)
link2 = browser.html
html_link2 = bs(link2, 'html.parser')
sch_hem = html_link2.find("img", class_="wide-image")["src"]
schiaparelli_png = astrogeology_url + sch_hem
scrape_dict["schiaparelli_png"] = schiaparelli_png

In [84]:
print(schiaparelli_png)

https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [85]:
link_pic3= "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
browser.visit(link_pic3)
link3 = browser.html
html_link3 = bs(link3, 'html.parser')
syrt_hem = html_link3.find("img", class_="wide-image")["src"]
syrtis_png = astrogeology_url + syrt_hem
scrape_dict["syrtis_png"] = syrtis_png

In [86]:
print(syrtis_png)

https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [87]:
link_pic4= "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"
browser.visit(link_pic4)
link4 = browser.html
html_link4 = bs(link4, 'html.parser')
valles_hem = html_link4.find("img", class_="wide-image")["src"]
valles_png = astrogeology_url + valles_hem
scrape_dict["valles_png"] = valles_png

In [88]:
print(valles_png)

https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [9]:
def scrape():
    browser = init_browser()
    scrape_dict = {}
    nasa_url = "https://mars.nasa.gov/news/"
    browser.visit(nasa_url)
    nasa_html = browser.html
    nasa_soup=bs(nasa_html, 'html.parser')
    titles = []
    results = nasa_soup.find_all('div',class_="content_title")  
    latest_title=results[0].text
    scrape_dict["latest_title"] = latest_title
    paragraph = []
    paras = nasa_soup.find_all('div', class_='rollover_description_inner')
    latest_paragraph = paras[0].text
    scrape_dict["latest_paragraph"] = latest_paragraph
    featured_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(featured_image)
    mars_browser = browser.html
    featured_mars=bs(mars_browser,'html.parser')
    featured_path = featured_mars.find("a","fancybox")["data-fancybox-href"]
    jpl_link = "https://www.jpl.nasa.gov/" + featured_path
    scrape_dict["jpl_link"]= jpl_link

    twitter_url = "https://twitter.com/marswxreport?lang=en"
    weather_response = requests.get(twitter_url)
    weather_tweets = bs(weather_response.text,"html.parser")
    #first weather tweet for mars 
    latest_weather = weather_tweets.find_all("p",class_="TweetTextSize")[0].text.strip()
    scrape_dict["latest_weather"] = latest_weather
    #mars facts
    facts_url = "https://space-facts.com/mars/"
    facts_response = requests.get(facts_url)
    facts_soup = bs(facts_response.text,"html.parser")
    table = facts_soup.find_all('table')[0]
    facts_list = pd.read_html(str(table))[0]
    df = facts_list.rename(index = str, columns={0:"Description", 1:"Facts"})
    html_table = df.to_html()
    scrape_dict["facts_table"] = html_table

    #hemispheres
    # link_pic1 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
    # cerberus_response = requests.get(link_pic1)
    # cerberus_soup = bs(cerberus_response.text,"html.parser")
    # cerberus_soup.find_all("img",class_="wide-image")
    # cerberus_img = "https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg"
    # scrape_dict["cerberus_img"] = cerberus_img
    link_pic1 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
    browser.visit(link_pic1)
    link1 = browser.html
    html_link1 = bs(link1,'html.parser')
    cer_hem = html_link1.find("img", class_="wide-image")["src"]
    astrogeology_url = "https://astrogeology.usgs.gov/"
    cerberus_png = astrogeology_url + cer_hem
    scrape_dict["cerberus_png"] = cerberus_png

    link_pic2 = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
    browser.visit(link_pic2)
    link2 = browser.html
    html_link2 = bs(link2, 'html.parser')
    sch_hem = html_link2.find("img", class_="wide-image")["src"]
    schiaparelli_png = astrogeology_url + sch_hem
    scrape_dict["schiaparelli_png"] = schiaparelli_png

    link_pic3= "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
    browser.visit(link_pic3)
    link3 = browser.html
    html_link3 = bs(link3, 'html.parser')
    syrt_hem = html_link3.find("img", class_="wide-image")["src"]
    syrtis_png = astrogeology_url + syrt_hem
    scrape_dict["syrtis_png"] = syrtis_png
   
    link_pic4= "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"
    browser.visit(link_pic4)
    link4 = browser.html
    html_link4 = bs(link4, 'html.parser')
    valles_hem = html_link4.find("img", class_="wide-image")["src"]
    valles_png = astrogeology_url + valles_hem
    scrape_dict["valles_png"] = valles_png
    

    hemisphere_image_urls = [
        {"title":"Cerberus Hemisphere","img_url":cerberus_png},
        {"title":"Schiaparelli Hemisphere","img_url":schiaparelli_png},
        {"title":"Syrtis Major","img_url":syrtis_png},
        {"title":"Valles Marineris","img_url":valles_png}    
    ]
 
    scrape_dict["hemisphere_image_urls"] = hemisphere_image_urls


    collection.insert_one(scrape_dict)


    return scrape_dict

In [12]:
scrape()

{'_id': ObjectId('5adc1544813d43bf61cdf761'),
 'cerberus_png': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
 'facts_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Facts</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>